<a href="https://colab.research.google.com/github/arijitdas0211/ML_Python/blob/main/Google_Colab_Emotion_Detection_Live_Image_Capture.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install deepface
!pip install opencv-python
!pip install matplotlib

In [ ]:
try:
  from IPython.display import display, Javascript, Image
  from google.colab.output import eval_js
  from base64 import b64decode
  import cv2
  from deepface import DeepFace
  import matplotlib.pyplot as plt
  import random
  import string
except Exception as err:
  print("Please install all dependencies first...\n", str(err))

In [ ]:
def take_photo(filename, quality):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Click me to Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename

In [ ]:
def generate_random_filename(length: int = 10, extension: str = "jpg") -> str:
    characters = string.ascii_letters + string.digits
    random_string = "".join(random.choice(characters) for _ in range(length))
    if extension:
        if "." in extension:
            pieces = extension.split(".")
            last_extension = pieces[-1]
            extension = last_extension
        return f"{random_string}.{extension}"
    return random_string

In [ ]:
try:
  filename = take_photo((f'{generate_random_filename()}'), 1)
  print('Saved to {}'.format(filename))
  display(Image(filename))
except Exception as err:
  print(str(err))

In [ ]:
try:
  img = cv2.imread(f'/content/{filename}')
  plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
  print('Image Found...\n')
except Exception as err:
  print('Please Capture and upload image first...\n',str(err))

In [ ]:
try:
  predictions = DeepFace.analyze(img_path = img, actions = ['emotion'], detector_backend = 'mtcnn')
  print(f"\nAccording to Deepface, the face emotion is:- {predictions[0]['dominant_emotion'].title()} \n")
  print(f"All emotion parameters are:-\n{predictions[0]}")
except Exception as err:
  print(err)
  print('Ohh! Face is not detected clearly. Please try to capture again...!!')